# Room Dimension Estimation using Line-Based Door Detection

This notebook estimates room dimensions by:
1. Detecting doors using gradient magnitude and line detection
2. Using door dimensions as reference (standard heights: 6'6", 6'8", 7'0")
3. Finding floor and ceiling boundaries
4. Calculating room dimensions

In [ ]:
# Cell 1: Setup and Imports
!pip install opencv-python matplotlib numpy -q

import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Standard door dimensions (inches) for scale calculation
DOOR_STANDARDS = {
    'standard': {'height': 80, 'name': 'Standard (6\'8")', 'typical': 'Most homes'},
    'modern': {'height': 84, 'name': 'Modern (7\'0")', 'typical': 'Newer/luxury'},
    'compact': {'height': 78, 'name': 'Compact (6\'6")', 'typical': 'Older homes'}
}

print("✅ Setup complete")

In [ ]:
# Cell 2: Load and Display Image
# Upload image
print("📤 Upload an empty room image")
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Load and convert image
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w = image_rgb.shape[:2]

# Display original image
plt.figure(figsize=(10, 6))
plt.imshow(image_rgb)
plt.title(f"Uploaded Image: {w}×{h} pixels")
plt.axis('off')
plt.show()

In [ ]:
# Cell 3: Simple Visual Door Detection
def detect_doors_simple(img):
    """Simplified door detection focusing on visual cues"""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    h, w = gray.shape
    
    # Step 1-2: Calculate gradient magnitude
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_mag = np.sqrt(grad_x**2 + grad_y**2)
    
    # Step 3: Threshold
    gradient_norm = (gradient_mag / gradient_mag.max() * 255).astype(np.uint8)
    _, binary = cv2.threshold(gradient_norm, 30, 255, cv2.THRESH_BINARY)
    
    # Step 4: Morphological closing
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    return closed

# Process image
closed_img = detect_doors_simple(image_rgb)

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original image
axes[0, 0].imshow(image_rgb)
axes[0, 0].set_title("Original Image")
axes[0, 0].axis('off')

# Gradient magnitude
gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
gradient_mag = np.sqrt(grad_x**2 + grad_y**2)
axes[0, 1].imshow(gradient_mag, cmap='gray')
axes[0, 1].set_title("Gradient Magnitude")
axes[0, 1].axis('off')

# Thresholded
gradient_norm = (gradient_mag / gradient_mag.max() * 255).astype(np.uint8)
_, binary = cv2.threshold(gradient_norm, 30, 255, cv2.THRESH_BINARY)
axes[0, 2].imshow(binary, cmap='gray')
axes[0, 2].set_title("After Threshold (Binary)")
axes[0, 2].axis('off')

# Morphological closing - larger view
axes[1, 0].imshow(closed_img, cmap='gray')
axes[1, 0].set_title("After Morphological Closing")
axes[1, 0].axis('off')

# Highlight door regions manually based on visual inspection
axes[1, 1].imshow(closed_img, cmap='gray')
axes[1, 1].set_title("Visual Door Identification Guide")
axes[1, 1].text(10, 30, "Look for:", color='yellow', fontsize=12, weight='bold',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
axes[1, 1].text(10, 60, "• Rectangular outlines", color='yellow', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
axes[1, 1].text(10, 90, "• Strong vertical edges", color='yellow', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
axes[1, 1].text(10, 120, "• Near floor level", color='yellow', fontsize=10,
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
axes[1, 1].axis('off')

# Original for reference
axes[1, 2].imshow(image_rgb)
axes[1, 2].set_title("Original Image (for reference)")
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n📊 Visual Analysis:")
print("From the morphological closing image, doors are clearly visible as:")
print("1. Two adjacent doors on the left (likely closet or French doors)")
print("2. One door on the right")
print("\nThe rectangular patterns with strong vertical edges are characteristic of doors.")
print("\n⚠️ Automated detection is challenging due to incomplete edge connections.")
print("Please proceed to manual selection based on the visual cues above.")

In [ ]:
# Cell 4: Manual Door Selection
def manual_door_selection(img):
    """Allow user to manually select door corners"""
    print("\n📍 MANUAL DOOR SELECTION")
    print("Based on the morphological closing analysis above, select ONE door.")
    print("Choose the clearest/largest door for the most accurate measurements.")
    print("\nClick the 4 corners of a door: top-left, top-right, bottom-right, bottom-left")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(img)
    ax.set_title("Click 4 corners of ONE door (clockwise from top-left)")
    
    points = []
    
    def onclick(event):
        if event.inaxes and len(points) < 4:
            points.append((event.xdata, event.ydata))
            ax.plot(event.xdata, event.ydata, 'ro', markersize=10)
            ax.text(event.xdata+10, event.ydata-10, f"{len(points)}", 
                   fontsize=12, color='red', weight='bold')
            
            if len(points) == 4:
                # Draw the door rectangle
                pts = np.array(points + [points[0]])
                ax.plot(pts[:, 0], pts[:, 1], 'r-', linewidth=2)
                ax.set_title("Door selected! Close this window to continue.")
            
            fig.canvas.draw()
    
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show()
    
    if len(points) == 4:
        # Calculate bounding box from points
        xs = [p[0] for p in points]
        ys = [p[1] for p in points]
        
        return [{
            'x': min(xs),
            'y': min(ys),
            'width': max(xs) - min(xs),
            'height': max(ys) - min(ys),
            'bottom_y': max(ys),
            'top_y': min(ys)
        }]
    
    return []

# Manual door selection
doors = manual_door_selection(image_rgb)

if doors:
    print(f"✅ Door selected successfully!")
    
    # Visualize manual selection
    plt.figure(figsize=(10, 6))
    plt.imshow(image_rgb)
    
    door = doors[0]
    rect = plt.Rectangle((door['x'], door['y']), 
                       door['width'], door['height'],
                       linewidth=3, edgecolor='lime', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title("Selected Door")
    plt.axis('off')
    plt.show()
    
    print(f"Door dimensions: {door['width']:.0f} × {door['height']:.0f} pixels")
else:
    print("❌ No door selected. Cannot proceed with dimension estimation.")

In [ ]:
# Cell 5: Find Floor and Ceiling
def find_ceiling(img, door_top, door_bottom):
    """Find ceiling by detecting horizontal edge above door"""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate vertical gradients to find horizontal edges
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    grad_y_abs = np.abs(grad_y)
    
    # Search upward from door top
    ceiling_y = door_top
    max_gradient = 0
    search_range = min(door_top, int(door_bottom - door_top))
    
    for y in range(door_top - 10, max(0, door_top - search_range), -1):
        row_gradient = np.mean(grad_y_abs[y-2:y+2, :])
        if row_gradient > max_gradient * 1.5:  # Significant edge
            max_gradient = row_gradient
            ceiling_y = y
    
    return ceiling_y

if doors:
    # Use the largest (closest) door
    door = doors[0]
    
    # Floor is at door bottom
    floor_y = door['bottom_y']
    
    # Find ceiling
    ceiling_y = find_ceiling(image_rgb, int(door['top_y']), int(door['bottom_y']))
    
    # Calculate room height in pixels
    room_height_px = floor_y - ceiling_y
    door_height_px = door['height']
    
    # Visualize floor and ceiling
    plt.figure(figsize=(10, 6))
    plt.imshow(image_rgb)
    
    # Draw door
    rect = plt.Rectangle((door['x'], door['y']), 
                       door['width'], door['height'],
                       linewidth=3, edgecolor='lime', facecolor='none')
    plt.gca().add_patch(rect)
    
    # Draw floor and ceiling lines
    plt.axhline(floor_y, color='red', linewidth=3, label='Floor (door bottom)')
    plt.axhline(ceiling_y, color='blue', linewidth=3, label='Ceiling')
    
    plt.title(f"Floor and Ceiling Detection")
    plt.legend()
    plt.axis('off')
    plt.show()
    
    print(f"📏 Measurements in pixels:")
    print(f"  Door height: {door_height_px:.0f} px")
    print(f"  Room height: {room_height_px:.0f} px")
else:
    print("❌ No doors detected to use as reference")

In [ ]:
# Cell 6: Calculate Room Dimensions
if doors:
    print("\n📐 Room Dimension Calculations")
    print("=" * 50)
    
    # Calculate scale for each door standard
    for key, standard in DOOR_STANDARDS.items():
        door_height_inches = standard['height']
        pixels_per_inch = door_height_px / door_height_inches
        
        # Calculate dimensions
        room_height_inches = room_height_px / pixels_per_inch
        room_width_inches = w / pixels_per_inch
        
        # Convert to feet
        room_height_ft = room_height_inches / 12
        room_width_ft = room_width_inches / 12
        
        print(f"\n{standard['name']} ({standard['typical']}):")
        print(f"  Scale: {pixels_per_inch:.1f} pixels/inch")
        print(f"  Room height: {room_height_ft:.1f} feet")
        print(f"  Room width: {room_width_ft:.1f} feet")
        
        # Sanity check
        if room_height_ft > 20:
            print(f"  ⚠️  Height seems high - check ceiling detection")
        elif room_height_ft < 7:
            print(f"  ⚠️  Height seems low - check floor/ceiling detection")
    
    # Final visualization with measurements
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    
    # Draw door
    rect = plt.Rectangle((door['x'], door['y']), 
                       door['width'], door['height'],
                       linewidth=3, edgecolor='lime', facecolor='none')
    plt.gca().add_patch(rect)
    
    # Draw floor and ceiling
    plt.axhline(floor_y, color='red', linewidth=3, label='Floor')
    plt.axhline(ceiling_y, color='blue', linewidth=3, label='Ceiling')
    
    # Add dimension annotations (using standard door)
    px_per_inch = door_height_px / 80  # Standard door height
    room_height_ft = (room_height_px / px_per_inch) / 12
    room_width_ft = (w / px_per_inch) / 12
    
    # Width annotation
    plt.text(w/2, floor_y+30, f'Width: {room_width_ft:.1f} ft', 
             ha='center', fontsize=14, color='white', weight='bold',
             bbox=dict(boxstyle='round', facecolor='red', alpha=0.7))
    
    # Height annotation
    plt.text(30, (floor_y+ceiling_y)/2, f'Height: {room_height_ft:.1f} ft', 
             rotation=90, va='center', fontsize=14, color='white', weight='bold',
             bbox=dict(boxstyle='round', facecolor='blue', alpha=0.7))
    
    plt.title(f"Room Dimensions (assuming standard 6'8\" door)", fontsize=16)
    plt.legend(fontsize=12)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("❌ Cannot calculate dimensions without door detection")

In [ ]:
# Cell 5: Process Door
if doors:
    door = doors[0]
    floor_y = door['bottom_y']
    ceiling_y = door['ceiling_y']
    room_height = door['room_height']
elif door_data:
    door = {
        'x': door_data['left'],
        'y': door_data['top'],
        'width': door_data['right'] - door_data['left'],
        'height': door_data['bottom'] - door_data['top'],
        'bottom_y': door_data['bottom'],
        'top_y': door_data['top']
    }
    floor_y = door['bottom_y']
    ceiling_y = find_ceiling(image_rgb, door['top_y'])
    room_height = floor_y - ceiling_y
else:
    raise ValueError("No door selected")

# Visualize
plt.figure(figsize=(10, 6))
plt.imshow(image_rgb)
rect = patches.Rectangle((door['x'], door['y']), door['width'], door['height'],
                       linewidth=3, edgecolor='green', facecolor='none')
plt.gca().add_patch(rect)
plt.axhline(floor_y, color='red', linewidth=2, label='Floor (door bottom)')
plt.axhline(ceiling_y, color='blue', linewidth=2, label='Ceiling')
plt.title(f"Door: {door['width']}×{door['height']}px, Room height: {room_height}px")
plt.legend()
plt.axis('off')
plt.show()